In [ ]:
import pickle

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
WIN = 'D:/'
MAC = '/Volumes/Transcend2/'

osp = MAC

### Modify mzTab files based on the previously obtained models

In [ ]:
with open('D:/pymz/mzmine/vlmodel.txt', 'rb') as file:  
    mvl = pickle.load(file)
with open('D:/pymz/mzmine/malmodel.txt', 'rb') as file:  
    mmal = pickle.load(file)

In [ ]:
#for modifying the samples files
vl_samples = os.path.join(osp, 'pymz/mzmine/221119/vl')
vl_samples_new =  os.path.join(osp,'pymz/mzmine/221119/vl_changed_rt')
mal_samples =  os.path.join(osp,'pymz/mzmine/221119/mal')
mal_samples_new =  os.path.join(osp,'pymz/mzmine/221119/mal_changed_rt')

In [ ]:
#for modifying the standard files
output_dir = os.path.join(osp, 'pymz/mzmine/221119/standards/')
output_dir_new = os.path.join(osp, 'pymz/mzmine/221119/standards_changed_rt/')

In [ ]:
from metab_utils import modify_rtdrift_in_mztab

modify_rtdrift_in_mztab(output_dir, output_dir_new, name_tag = 'Mal*.mzTab', model = mmal, standards = True)
modify_rtdrift_in_mztab(output_dir, output_dir_new, name_tag = 'VL*.mzTab', model = mvl, standards = True)

modify_rtdrift_in_mztab(mal_samples, mal_samples_new, name_tag = '*.mzTab', model = mmal, standards = False)
modify_rtdrift_in_mztab(vl_samples, vl_samples_new, name_tag = '*.mzTab', model = mvl, standards = False)

### Modify csv files from GNPS based on the previously obtained models

In [ ]:
vl_ms2 = os.path.join(osp, 'pymz/mzmine/210120/mzmine_frag/vl/frag_pos_vl_quant.csv')
vl_ms2_new = os.path.join(osp, 'pymz/mzmine/210120/mzmine_frag/vl_changed_rt/frag_pos_vl_quant.csv')
mal_ms2 = os.path.join(osp, 'pymz/mzmine/210120/mzmine_frag/mal/frag_pos_mal_quant.csv')
mal_ms2_new = os.path.join(osp, 'pymz/mzmine/210120/mzmine_frag/mal_changed_rt/frag_pos_mal_quant.csv')

In [ ]:
modify_rtdrift_in_csv(vl_ms2, vl_ms2_new, mvl)
modify_rtdrift_in_csv(mal_ms2, mal_ms2_new, mmal)

### Modify raw mzXML files based on the previously obtained models

In [ ]:
from metab_utils import modify_rtdrif_in_mzxml
path_to_mzxmlfiles = '/Volumes/Transcend/PhD/Metabolomics_datasets/VL/Positive/modified/'
files = os.listdir(path_to_mzxmlfiles)
for filename in files:
    print(filename)
    modify_rtdrif_in_mzxml(path_to_mzxmlfiles, filename, mvl)

In [ ]:
mal = pd.read_csv('/Users/anamaria/Documents/pymz/mzmine/221119/mal/topfeatures_filled.txt', sep = '\t')
mal_sig = mal[mal['adj-p'] < 0.05]

vl = pd.read_csv('/Users/anamaria/Documents/pymz/mzmine/221119/vl/topfeatures_filled.txt', sep = '\t')
vl_sig = vl[vl['adj-p'] < 0.05]
zik = pd.read_csv('/Users/anamaria/Documents/pymz/mzmine/221119/zik/topfeatures_filled.txt', sep = '\t')
zik_sig = zik[zik['adj-p'] < 0.05]

In [ ]:
def change_rt_in_df(df, model):
    x = []
    for _,row in df.iterrows():

        old_rt = row['row retention time']
        new_rt,_ = model.predict(np.array([[old_rt]]))
        new_rt = new_rt[0][0]
        new_rt += old_rt
        row['row retention time'] = new_rt
        x.append(row)
    new_df = pd.DataFrame(x)
    return new_df

In [ ]:
mal_new = change_rt_in_df(mal, mmal)
vl_new = change_rt_in_df(vl, mvl)

In [ ]:
vl_new.to_csv(r'/Users/anamaria/Documents/pymz/mzmine/221119/vl/topfeatures_filled_changed_rt.txt', header=True, index=None, sep='\t', mode='w')
mal_new.to_csv(r'/Users/anamaria/Documents/pymz/mzmine/221119/mal/topfeatures_filled_changed_rt.txt', header=True, index=None, sep='\t', mode='w')

In [ ]:
new_rt,_ = model.predict(np.array([[old_rt]]))
                    new_rt = new_rt[0][0]
                    new_rt += old_rt
                    tokens[2] = new_rt

In [ ]:
def get_rt_is_in_hmdb(root, mz, tolerance = 0.05):
    lista = []
    stringtag = '{http://www.hmdb.ca}'
    for metabolite in root.findall(stringtag+'metabolite'):
            name = metabolite.find(stringtag+'name').text
            mass = metabolite.find(stringtag+'monisotopic_molecular_weight').text
            if isinstance(mass, str):
                mass = float(mass)
                upper_mass = mass+tolerance
                lower_mass = mass-tolerance
                if mz > lower_mass and mz < upper_mass:
                    lista.append(name)
    return lista

names = {}
for _,row in with_mod.iterrows():
    mz = row['row m/z']
    cid = row['custom_id']
    for adduct in adducts_pos_mod:
        mz = adducts_pos_mod[adduct] + mz
        hf = check_peak_is_in_hmdb(root, mz, 0.01)
        names[int(cid)] = hf
        
   